In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import urllib, json
import re
from sqlalchemy import create_engine
import psycopg2
import numpy as np

#get latest upload time
con = psycopg2.connect(database="udw_workplace", user="menghezhang", password="*****", host="*****.com", port="****")
cur = con.cursor()
cur.execute(
'''
select a.*,b.fullname,b.division from 
(select distinct account_id,account_name,acct_owner from analytics.sf_Scd_accounts where active_ptc is not null and active_ptc <> 0 and is_active and not isdeleted ) as a 
left join 
(select distinct salesforce_id,firstname || ' ' || lastname as fullname,division from analytics.sf_scd_users where is_active and not isdeleted) as b
on a.acct_owner=b.salesforce_id
order by a.account_name
'''  )
con.commit()
rows = cur.fetchall()
# account_name=pd.DataFrame(data=rows).rename(columns={0:'sfid',1:'account'})

In [6]:
account_name

,account_id,account_name,acct_owner,fullname,division
0,0018000001IoIEjAAN,1-800-Flowers.com,0052H00000CLaszQAD,Jacob Meyer,LA
1,0013400001T5TwmAAF,3M,0052H00000CLbI4QAL,Alex Lang,STG
2,0018000001DSHHKAA5,55places.com,0052H00000CLX38QAH,Kelly Kevin,SMB
3,0013400001Wtyk9AAB,ABB,0052H00000CLXgEQAX,Stefanie Ross,LA
4,0013400001KGwgQAAT,AB InBev,0052H00000CLXCnQAP,Morgan Crouch,None
...,...,...,...,...,...
279,0013400001SYcRCAA1,Woodruff Sawyer,0052H00000CLWymQAH,Jenna Baagoe,SMB
281,0012H00001ZNGctQAH,Zamora,0053400000BnVjOAAV,Emma Hilder,SMB
282,0013400001T5Ts4AAF,ZEBRA TECHNOLOGIES CORPORATION,0052H00000CLbQIQA1,Stephanie Lowry,LA
283,0013400001OYyIeAAL,Zendesk,0052H00000CecKRQAZ,Katherine Zera,LA


In [2]:
field_names = [i[0] for i in cur.description]
account_name=pd.DataFrame(data=rows,columns=field_names)

## remove the account that can't be found in Indeed

In [3]:
def account_not_found_remove ():
    a=[]
    for i in range(account_name.shape[0]):
        url='https://www.indeed.com/cmp/'+account_name['account_name'][i]
        r = requests.get(url)
        parsed_html = bs(r.content, features="html.parser")
        title = parsed_html.find('title').string
        if 'Not found' in title:
            a.append(i)
    return a

In [4]:
p=account_not_found_remove()
account_name=account_name.drop(p)

In [5]:
account_name

,account_id,account_name,acct_owner,fullname,division
0,0018000001IoIEjAAN,1-800-Flowers.com,0052H00000CLaszQAD,Jacob Meyer,LA
1,0013400001T5TwmAAF,3M,0052H00000CLbI4QAL,Alex Lang,STG
2,0018000001DSHHKAA5,55places.com,0052H00000CLX38QAH,Kelly Kevin,SMB
3,0013400001Wtyk9AAB,ABB,0052H00000CLXgEQAX,Stefanie Ross,LA
4,0013400001KGwgQAAT,AB InBev,0052H00000CLXCnQAP,Morgan Crouch,None
...,...,...,...,...,...
279,0013400001SYcRCAA1,Woodruff Sawyer,0052H00000CLWymQAH,Jenna Baagoe,SMB
281,0012H00001ZNGctQAH,Zamora,0053400000BnVjOAAV,Emma Hilder,SMB
282,0013400001T5Ts4AAF,ZEBRA TECHNOLOGIES CORPORATION,0052H00000CLbQIQA1,Stephanie Lowry,LA
283,0013400001OYyIeAAL,Zendesk,0052H00000CecKRQAZ,Katherine Zera,LA


## transform URL to Json object

In [7]:
def fixed_json(url):
    r = requests.get(url)
    parsed_html = bs(r.content, features="html.parser")
    script_tags = parsed_html.body.find_all('script')
    for script_tag in script_tags:
        text = script_tag.text
        if 'totalJobCount'.lower() in text.lower():
            b=text[text.index("{") + 1:text.rindex("}")]
            c="{"+b+"}"
            regex = re.compile(r'\\(?![/u"])')
            fixed = regex.sub(r"\\\\", c)
            fixed_json = json.loads(fixed)
            return fixed_json    

## grab jobs of each account

In [8]:
def jobs_per_account(account_name):
    url = 'https://www.indeed.com/cmp/'+account_name+'/jobs'
    jobcount = int(fixed_json(url)['jobList']['totalJobCount'])
    if jobcount != 0:
        job_page = jobcount//100+1
        final = pd.DataFrame()
        for i in range(job_page):
            url = 'https://www.indeed.com/cmp/'+account_name+'/jobs?q=&l=&start='+str(i*100)+'#cmp-skip-header-desktop'
            joblist = fixed_json(url)['jobList']['jobs']
            jobs=pd.DataFrame.from_dict(data=joblist)
            final = pd.concat([final,jobs], axis=0,sort=True)
            final=final[['companyName','title','formattedRelativeTime','jobKey','location','urgentHire','url']]
            final=final.drop_duplicates(subset='jobKey', keep="first")
            final['url']="http://www.indeed.com/rc/clk?jk="+str(final['jobKey'])
        return final

In [9]:
def total_jobs():
    total_jobs = pd.DataFrame()
    for i in account_name.index.tolist():
        per_account = jobs_per_account(account_name['account_name'][i])
        total_jobs = pd.concat([total_jobs,per_account], axis=0,sort=True)
    return total_jobs

In [10]:
M=total_jobs()

In [11]:
M

,companyName,formattedRelativeTime,jobKey,location,title,urgentHire,url
0,1-800-Flowers,20 days ago,cb7ea45eb322a648,"Melrose Park, IL",Social Media Manager,False,http://www.indeed.com/rc/clk?jk=0 cb7ea45e...
1,1-800-Flowers,3 days ago,580534b24a3603c0,"Melrose Park, IL",Project Management Assistant,False,http://www.indeed.com/rc/clk?jk=0 cb7ea45e...
2,1-800-Flowers,20 days ago,cad40b1491f3876f,"Melrose Park, IL",Graphic Designer,False,http://www.indeed.com/rc/clk?jk=0 cb7ea45e...
3,1-800-Flowers,1 day ago,6105341151bff665,"New York, NY","Marketing Coordinator, Paid Search",False,http://www.indeed.com/rc/clk?jk=0 cb7ea45e...
4,1-800-Flowers,9 days ago,a2787f834fd1f73f,"Melrose Park, IL",Account Manager - Simply Chocolate,False,http://www.indeed.com/rc/clk?jk=0 cb7ea45e...
...,...,...,...,...,...,...,...
8,Zocdoc,30+ days ago,0504c223bbc17b6b,"Scottsdale, AZ",Network Engineer,False,http://www.indeed.com/rc/clk?jk=0 ee11a7ac...
9,Zocdoc,3 days ago,8862b4e5fb8d4fb5,"New York, NY",Senior Product Manager - Mobile,False,http://www.indeed.com/rc/clk?jk=0 ee11a7ac...
10,Zocdoc,30+ days ago,b35265f01446f10f,"New York, NY",Senior Product Manager,False,http://www.indeed.com/rc/clk?jk=0 ee11a7ac...
11,Zocdoc,30+ days ago,3638f449066fa008,"New York, NY",Senior Software Engineer,False,http://www.indeed.com/rc/clk?jk=0 ee11a7ac...


In [991]:
M=M.rename(columns={"companyName": "Account_Name", "formattedRelativeTime": "Indeed_Posted_Date",'jobKey':'Indeed_Job_Id',
                 "location":'Location',"title":"Indeed_Job_Title","urgentHire":"Urgent_Hire","url":"URL"})

In [992]:
M

,Account_Name,Indeed_Posted_Date,Indeed_Job_Id,Location,Indeed_Job_Title,Urgent_Hire,Indeed_URL
0,1-800-Flowers.com,14 days ago,a4af610be8b342e9,"Carle Place, NY","Director, Customer Service and Call Center Ope...",False,http://www.indeed.com/rc/clk?jk=0 a4af610b...
1,1-800-Flowers.com,30+ days ago,1230daeba18f6a8a,"Melrose Park, IL",Human Resources Generalist,False,http://www.indeed.com/rc/clk?jk=0 a4af610b...
2,1-800-Flowers.com,14 days ago,64c037e2131f008b,"Carle Place, NY","Senior Manager, Customer Service and Call Cent...",False,http://www.indeed.com/rc/clk?jk=0 a4af610b...
3,1-800-Flowers.com,16 days ago,7d7cb20aa2a96096,"Carle Place, NY",Corporate Social Responsibility Director,False,http://www.indeed.com/rc/clk?jk=0 a4af610b...
4,1-800-Flowers.com,1 day ago,9cfa6fa181fbcd79,"Carle Place, NY","Director, Leadership Training & Talent Develop...",False,http://www.indeed.com/rc/clk?jk=0 a4af610b...
...,...,...,...,...,...,...,...
1,ZocDoc,30+ days ago,7abde199d3f1e926,"New York, NY",Director of Data,False,http://www.indeed.com/rc/clk?jk=0 847bd6ce7...
2,ZocDoc,30+ days ago,b35265f01446f10f,"New York, NY",Senior Product Manager,False,http://www.indeed.com/rc/clk?jk=0 847bd6ce7...
3,ZocDoc,30+ days ago,88ec8ffe67c368e0,"Phoenix, AZ",Customer Service Technician,False,http://www.indeed.com/rc/clk?jk=0 847bd6ce7...
4,ZocDoc,16 days ago,adb5a8e70682c578,"New York, NY",Senior Software Engineer - Full Stack,False,http://www.indeed.com/rc/clk?jk=0 847bd6ce7...


In [878]:
url = 'https://www.indeed.com/cmp/Abb/jobs'
jobcount = fixed_json(url)['jobList']['totalJobCount']
job_page = jobcount//100+1
job_page

3

In [384]:
import requests
from bs4 import BeautifulSoup
def job_count(account_name):
    url = 'https://www.indeed.com/cmp/'+account_name
    r = requests.get(url)
    parsed_html = BeautifulSoup(r.content, features="html.parser")
    script_tags = parsed_html.body.find_all('script')
    for script_tag in script_tags:
        text = script_tag.text
        if 'totalJobCount'.lower() in text.lower():
            try:
                job_count=int(text.split("\"totalJobCount\":")[1].split("}")[0])
                return job_count
            except:
                pass
#                 employee_range=text.split("\"employeeRange\":\"")[1].split("\",\"")[0]
#                 company_info=company_info.append({'account_name':account_name.loc[i][0],
#                                             'job_count':job_count,
#                                             'employee_range':employee_range}, ignore_index=True)
            

In [852]:
def job_info (account_name,job_page):
    df = pd.DataFrame()
    for p in range(0, job_page*10, 10):
        page = requests.get("https://www.indeed.com/jobs?q=company%3A"+account_name+"&start="+str(p))
        soup = bs(page.content, 'html.parser')
        targetElements = soup.findAll('div', attrs={'class' : 'row'})
        for elem in targetElements: 
            for a in elem.find_all('a', attrs={'data-tn-element':'companyName'}):
                company = a.text.strip()
            for a in elem.find_all('span', attrs={'class':'location'}):
                location = a.text.split(",")[0]
            for a in elem.find_all('a', attrs={'class':'jobtitle turnstileLink'}):
                job_title = a.attrs["title"]
            home_url = "http://www.indeed.com"
            for a in elem.find_all('a', attrs={'class':'jobtitle turnstileLink'}):
                job_link = "%s%s" % (home_url,a['href'])
            for a in elem.find_all('span', attrs={'class': 'date'}):
                job_date=a.getText()  
            for a in elem.find_all(name="span", attrs={"class":"salaryText"}):
                salary=a.getText() 
                
            df = df.append({'company':company
                                , 'location': location
                                , 'job_title': job_title
                                , 'job_link': job_link
                                ,'job_date': job_date
                                ,'salary':salary}, ignore_index=True)
    return df

In [993]:
M.to_excel('p.xlsx')

C:\Users\menghezhang\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://www.indeed.com/rc/clk?jk=0%20%20%20%20%20a4af610be8b342e9
1%20%20%20%20%201230daeba18f6a8a
2%20%20%20%20%2064c037e2131f008b
3%20%20%20%20%207d7cb20aa2a96096
4%20%20%20%20%209cfa6fa181fbcd79
5%20%20%20%20%204b7cdd29511a16eb
6%20%20%20%20%20ffdce664c995e274
7%20%20%20%20%201a577d3e4801fb83
8%20%20%20%20%202239ea4d556e9596
9%20%20%20%20%20696a21e1f6b9a673
10%20%20%20%2000f5ad4e419c0012
11%20%20%20%2051fcdbdac14b2bc7
12%20%20%20%20a7b632f08ddc74c8
13%20%20%20%20d4eb16a74463651d
14%20%20%20%2093b86fcba8e4b4e9
15%20%20%20%20161e1ebd2cbcddb2
16%20%20%20%204ee16c7e0472e516
17%20%20%20%2076e3979738ef6da2
18%20%20%20%20517e3d644181be7a
19%20%20%20%2080af143e021681d1
20%20%20%20%20db6c2a2304db6254
21%20%20%20%20ca9ba8380b95ebf2
22%20%20%20%203ec43158a340e527
23%20%20%20%207f2b28b09d54172d
24%20%20%20%209739a744e41d8663
25%20%20%20%20043fa76e2ecad784
26%20%20%20%20b47a8599f6f9a66f
27%20%

C:\Users\menghezhang\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://www.indeed.com/rc/clk?jk=0%20%20%20%20%2087acb64e508b413c
1%20%20%20%20%2075853a5f08bfc3f0
2%20%20%20%20%20430f8c2db961fd0f
3%20%20%20%20%2077cd92fab0e1dd7b
4%20%20%20%20%205d59ac9b8d1ca340
%20%20%20%20%20%20%20%20%20%20%20%20...%20%20%20%20%20%20%20
97%20%20%20%20ef2332d1c77f3fc8
98%20%20%20%206cc247d05d2ef171
99%20%20%20%20d0567e8fa79925db
0%20%20%20%20%200b92b088d253ef55
61%20%20%20%20474bc2a26c3c43f7
Name:%20jobKey,%20Length:%20801,%20dtype:%20object' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\menghezhang\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://www.indeed.com/rc/clk?jk=0%20%20%20%20%20c718c3b3b4c971c0
1%20%20%20%20%204850c9929ef86583
2%20%20%20%20%204487a5efbdc70d9a
3%20%20%20%20%2002bb7a22ad6e0a2b
4%20%20%20%20%2035cb25730172e85a
5%20%20%20%20%208d